## 1.1

Q: Determine if a string has all unique characters.

In [3]:
function all_unique(s) {
    return new Set(s).size == s.length;
}

In [4]:
assert = require('assert');
assert.equal(all_unique(''), true);
assert.equal(all_unique('a'), true);
assert.equal(all_unique('aa'), false);

## 1.2

Q: Given two strings determine if one is a permutation of the other.

The non-optimized solution is simple: compared the sorted strings. The sort will be $O(n\log{n})$, and the comparison will be $O(n)$, so the complete solution will be $O(n^2\log{n})$.

However, this is not the best possible runtime. The best possible runtime is $O(n^2)$ (we would need to interate through each string at least once, at least until a failure condition is met). Here is an optimized psuedocode. This algorithm is indeed $O(n^2)$.

```
function is_permutation(str1, str2):
    # Base cases.
    if |str1| != |str2| return False
    elif |str1| == 0 and |str2| == 0 return False

    m1 = {}  # hash map
    m1_keys = Set()

    # Build the string one character count.
    for char of str1:
        if char in m1_keys:
            m1[char] += 1
        else:
            m1_keys += {char}
            m1[char] = 1

    m2 = {}
    m2_keys = Set()
    # Build the string two character count.
    for char of str2:
        # If the string does not contain it.
        if char not in m1_keys:
            return False

        if char not in m2_keys:
            m2_keys += {char}
            m2[char] = 1
        else:
            m2[char] += 1
            if m2[char] > m1[char]:
                return false

    return true
```

Now tests and implementation.

In [10]:
assert = require('assert');
assert.equal(is_permutation('', ''), true);
assert.equal(is_permutation('a', 'aaa'), false);
assert.equal(is_permutation('abc', 'abc'), true);
assert.equal(is_permutation('abc', 'cba'), true);

In [9]:
function is_permutation(str1, str2) {
    if (str1.length !== str2.length)  return false;
    else if (str1.length === 0 & str2.length == 0)  return true;
    
    let str1_map = {};
    let str1_keys = new Set();
    for (let char of str1) {
        str1_keys.has(char) ? str1_map[char] += 1 : str1_map[char] = 0;
        str1_keys.add(char);        
    }
    
    let str2_map = {};
    let str2_keys = new Set();
    for (let char of str2) {
        if (!str1_keys.has(char))  return false;
        
        str2_keys.has(char) ? str2_map[char] += 1 : str2_map[char] = 0;
        str2_keys.add(char);
        
        if (str1_map[char] > str2_map[char])  return false;        
    }
    
    return true;
}

## 1.3

Q: Replace all spaces in a string with `%20`, in the manner appropriate for URL encoding (e.g. trim ending whitespace).

Again a brute force solution comes immediately:

In [34]:
function whiteparse(s) { return s.trimEnd().replace(/ /g, "%20"); }

In [37]:
assert = require('assert');
assert.equal(whiteparse(''), '');
assert.equal(whiteparse(' '), '');
assert.equal(whiteparse('a b c'), 'a%20b%20c');

Though you have to know some stdlib knick-knacks to implement this. TIL: `trim`, `trimEnd`, and `trimStart`; the equivalent of `strip`, `lstrip`, and `rstrip` in Python.

This is the readable brute-force solution. `trimEnd` is $O(n)$, and an efficient implementation of `replace` is $O(n)$, so this algorithm is $O(n^2)$.

Here's the same algorithm without built-ins...

```
function whiteparse(s):
    replace_idxs = []
    
    # First, skip the whitespace.
    i = |s| - 1
    while True:
        if s[i] == " ":
            continue
        else:
            break

    # Now find the indices of the characters needing replacement.
    for c in range(i):
        if s[c] == " ":
            replace_idxs.push(c)
            
    # Use this to determine the length of the output character array.
    c_a = Array(i + len(replace_idxs) * 3)
    
    i = 0
    for c in s:
        if s[c] == " ":
            c_a[i: i + 3] = "%20"
            i += 3
        else:
            c_a[i] = c
            i += 1
    
    return c_a.join("");
```

In [43]:
function whiteparse(s) {
    if (s.length === 0) return s;
    
    replace_idxs = [];
    
    i = s.length - 1;
    while (true) {
        if (i === 0) return ""
        else if (s[i] === " ") i -= 1
        else break
    }
    
    replace_idxs = [];
    for (let c of [...Array(i).keys()]) {
        c = parseInt(c);
        if (s.charAt(c) === " ") {
            replace_idxs.push(c);
        }
    }
    
    if (replace_idxs.length == 0) return s.slice(0, i);
    
    let c_a = Array(s.length - replace_idxs.length + replace_idxs.length * 3);
    
    i = 0;
    for (let c of s) {
        if (c === " ") {
            [c_a[i], c_a[i + 1], c_a[i +2]] = ["%", "2", "0"];
            i += 3;
        } else {
            c_a[i] = c;
            i += 1;
        }
    }
    
    return c_a.join("");
}

In [44]:
assert = require('assert');
assert.equal(whiteparse(''), '');
assert.equal(whiteparse(' '), '');
assert.equal(whiteparse('a b c'), 'a%20b%20c');

Comments on this problem:

* The first solution requires deep knowledge of the JavaScript stdlib. It's what you would do in production, but probably not impressive in an interview.
* There's a probably $O(n^2)$ operation which is $O(n)$ in amortized time. Instead of iterating over the strings a second time, use the known character-to-replace positions to chunk substrings. I had a hard time getting this solution to work right away. In retrospect, it's just an optimization on this brute-force solution, and I shouldn't have gone for it right away.
* The book uses this solution.
* Character arrays are useful!

## 1.4

Q: Check whether any of the permutations of a string is a palindrome or not.

The brute-force solution would be to iterate through all permutations of the string, stopping if we find a string that is a palindrome. However...this would be $O(n!)$.

Instead we can use the properties of a palindrome to check. Basically a string is a palindrome if it has a character count which is entirely even for every character, except for at most one odd count character (the palindrome pivot).

In [1]:
let _c = {}

In [28]:
function has_palindromic_permutation(s) {
    let countmap = {};
    function add(c) {
        (countmap.hasOwnProperty(c)) ? countmap[c] += 1 : countmap[c] = 1;
    }
    
    for (let c of s)  add(c);
    
    let n_odds = 0;
    for (let c of Object.keys(countmap)) {
        if (countmap[c] % 2 != 0) {
            if (n_odds > 0)  return false;
            else  n_odds += 1;
        }
    }
    
    return true;
}

In [33]:
assert.equal(has_palindromic_permutation('FFOO'), true)
assert.equal(has_palindromic_permutation('F'), true)
assert.equal(has_palindromic_permutation(''), true)
assert.equal(has_palindromic_permutation('AB'), false)

This algorithm is $O(kn)$, where $n$ is the length of the string and $k$ is the average number of unique characters in the string.

We could reduce the runtime to $O(n)$ if we performed the oddness bookkeeping directly in the body of the first loop. However this solution is much more readable and thus better outside of high-performance settings.

Comments:
* I initially goofed by making `add` an object method, causing it to pop up in the key iterator. Don't mix your objects and your data structures!

### 1.5

Q: *One-away edit distance* &mdash; Write a function that, given a string, check whether or not it is zero or one away in terms of edit distance.

Cmnnt: much easier than the DP problem of solving for shortest edit distance, which I struggled with some time ago.

Zero means equivalent. One means that we can perform an insert, delete, or swap operation to make the strings equivalent.

```python
function one_or_zero_away(s1, s2):
    if abs(|s1| - |s2|) > 1:
        return False
    
    s1, s2 = whitespacify(...s1, s2)
    
    have_fix_left = True
    
    for i in range(|s1|):
    
        if s1[i] != s2[i]:
            if not have_fix_left:
                return False
            
            if s1[i + 1] == s2[i]:
                s1 = s1[:i] + s1[i+1:]
                have_fix_left = False
            
            else if s1[i] == s2[i + 1]:
                s1 = s1[:i] + s1[i+1:]
                have_fix_left = False
                
            else:
                s2[i] = s1[i]
                have_fix_left = False
                
    return True
```

In [46]:
function one_or_zero_away(s1, s2) {
    if (Math.abs(s1.length - s2.length) > 1)  return false
    
    if (s1.length < s2.length) {
        s1 = s1.padEnd(s1.length - s2.length, " ");
    } else if (s1.length > s2.length) {
        s2 = s2.padEnd(s2.length - s1.length, " ");
    }
    
    let have_fix_left = true;
    for (let i of Object.keys([...Array(s1.length)])) {
            if (s1.charAt(i) !== s2.charAt(i)) {

            if (!have_fix_left) {
                return false;
            }

            if (s1.charAt(i + 1) === s2.charAt(i)) {
                s1 = s1.slice(0, i) + s1.slice(i + 1);
                have_fix_left = false;
            } else if (s1.charAt(i) === s2.charAt(i + 1)) {
                s1 = s1.slice(0, i) + s1.slice(i + 1);
                have_fix_left = false;
            } else {
                s1 = s1.slice(0, i) + s2.charAt(i) + s1.slice(i + 1);
                have_fix_left = false;
            }
        }
    }
    
    return true;
}

In [53]:
assert.equal(one_or_zero_away('aa', 'bbbb'), false);
assert.equal(one_or_zero_away('aa', 'aa'), true);
assert.equal(one_or_zero_away('a', 'aa'), true);
assert.equal(one_or_zero_away('bb', 'aa'), false);
assert.equal(one_or_zero_away('bba', 'bb'), true);
assert.equal(one_or_zero_away('abb', 'bb'), true);

This is $O(n)$, where $n$ is the length of the shorter string, as it iterates through the list of characters once.

Comment: this one went relatively well.

## 1.6

Q: Implement basic run-length encoding. E.g. transform `foobar` into `f1o2b1a1r1`.

We should be able to cook off an $O(n)$ solution immediately.

In [63]:
function rle(s) {
    if (s === "")  return ""
    
    let i = 1;
    let curr_length = 1;
    let curr_char = s.charAt(0);
    let out = Array(s.length);
    while (i < s.length) {
        if (s.charAt(i) === curr_char) {
            curr_length += 1;
        } else {
            out[i] = curr_char + curr_length;
            curr_char = s.charAt(i);
            curr_length = 1;
        }
        i += 1;
    }
    
    out[i] = curr_char + curr_length;
    return out.join("");
}

In [68]:
assert.equal(rle(''), '');
assert.equal(rle('a'), 'a1');
assert.equal(rle('abc'), 'a1b1c1');
assert.equal(rle('aaabbbcaa'), 'a3b3c1a2');

Note what we *didn't* do: we didn't concatenate to a string, because strings are immutable and we'd require $O(n^2)$ copies to mutate the string in place.

I used an array and joined the array in one operation at the end instead. The array was allocated to the same length (e.g. we used all $O(n)$ space) as the input string because this reduces the non-amortized running time ($O(n)$), as we don't need to deal with list length doubling. A small optimization.

The algorithm is ultimately $O(n)$. We iterate throug the string once for the $n$ term, and join the resultant substrings together in linear time.